Random DNA sample from https://www.bioinformatics.org/sms2/random_dna.html

Following tutorial on https://www.kaggle.com/thomasnelson/working-with-dna-sequence-data-for-ml/data

Bio Python help from https://biopython.org/wiki/SeqRecord

Questions

- One-hot vs. ordinal encoding -> no diff?
- Kmer counts = ideal because the lengths become the same
- Why are the genomes different lengths?
- How do you analyze the classifer? 
    - Which sequences are wrong or not, should you pass them in with labels? (the sequence id)

The following code will use a classifier to classify a genome as belonging to aedes or culex. It will use 100 samples from each.

In [42]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from Bio import SeqIO # FASTA reader
import numpy as np 

aedes_paths = ["/Users/alevenberg/Documents/research/VirLab/genomes/Full_Aedes/Dengue_virus.fasta", "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Aedes/Yellow_fever_virus.fasta", "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Aedes/Zika_virus.fasta"]
culex_paths = ["/Users/alevenberg/Documents/research/VirLab/genomes/Full_Culex/Japanese_encephalitis_virus.fasta", "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Culex/Tembusu_virus.fasta", "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Culex/Usutu_virus.fasta", "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Culex/West_nile_virus.fasta"]

aedes_path = aedes_paths[0]
culex_path = culex_paths[0]

# Transform sequences from files into an array
# For aedes 
aedes_sequences = []
for seq_record in SeqIO.parse(aedes_path, "fasta"):
    aedes_sequences.append(str(seq_record.seq))
    
# For culex
culex_sequences = []
for seq_record in SeqIO.parse(culex_path, "fasta"):
    culex_sequences.append(str(seq_record.seq))

# Pick the file with less sequences and trim the other one based on that
if(len(culex_sequences) > len(aedes_sequences)):
    culex_sequences = culex_sequences[0:len(aedes_sequences)]
elif(len(aedes_sequences) > len(culex_sequences)):
    aedes_sequences = aedes_sequences[0:len(culex_sequences)]

# Create labels
# Aedes - 0 
# Culex - 1
sequence_length = len(aedes_sequences)
aedes_classes = np.full(shape=sequence_length,fill_value=0)
culex_classes = np.full(shape=sequence_length,fill_value=1)

# Make dataframes
aedes_data = np.array(list(zip(aedes_sequences,aedes_classes)))
df_aedes = pd.DataFrame(data=aedes_data)
df_aedes.columns = ["sequence","class"]

culex_data = np.array(list(zip(culex_sequences,culex_classes)))
df_culex = pd.DataFrame(data=culex_data)
df_culex.columns = ["sequence","class"]

# Shape of both is (324, 2)

# Combine frames
frames = [df_aedes, df_culex]
full_df = df_aedes.append(df_culex)


# Transform into kmer counts 
KMERS = 3

def getKmersStr(sequence, size=KMERS):
    kmers = [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]
    return " ".join(kmers)

full_df['words'] = full_df.apply(lambda x: getKmersStr(x['sequence']), axis=1)
full_df = full_df.drop('sequence', axis=1)
labels = full_df.iloc[:, 0].values     
labels

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(human_texts)
aedes_kmers = cv.fit_transform(aedes_sequences).toarray() # Holds the kmers and their counts



array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0

In [39]:



# Check that is the same as the cartesian
from itertools import product

# Get all combinations of [a, c, g, t] and length of KMER
comb = product(['a','c','g','t'],repeat=KMERS) 
  
cart = []
# Print the obtained combinations 
for i in list(comb): 
    cart.append("".join(i))
    
# For aedes
aedes_sequences = list(map(getKmersStr, aedes_sequences))

cv = CountVectorizer()
aedes_kmers = cv.fit_transform(aedes_sequences).toarray() # Holds the kmers and their counts
tokens = cv.get_feature_names()
df_aedes = pd.DataFrame(data=aedes_kmers, columns=tokens)
df_aedes = df_aedes.loc[:, df_aedes.isin([' ','NULL',0]).mean() < .95] # Removes columns without kmers -> specifically ones that are not made of the correct bases: a,c,g,t
# Might need to change how to remove it

# Delete columns without a/c/g/t
droplist = [i for i in df_aedes.columns if any(c not in 'acgt' for c in i)]
df_aedes.drop(droplist,axis=1,inplace=True)

assert(len(df_aedes.columns) == len(cart)) # Make sure all possible KMERS exist

# For culex

aedes_sequences = list(map(getKmersStr, aedes_sequences))

cv = CountVectorizer()
culex_kmers = cv.fit_transform(culex_sequences).toarray() # Holds the kmers and their counts
tokens = cv.get_feature_names()
df_culex = pd.DataFrame(data=culex_kmers, columns=tokens)
df_culex = df_culex.loc[:, df_culex.isin([' ','NULL',0]).mean() < .95] # Removes columns without kmers -> specifically ones that are not made of the correct bases: a,c,g,t
# Might need to change how to remove it

# Delete columns without a/c/g/t
droplist = [i for i in df_culex.columns if any(c not in 'acgt' for c in i)]
df_culex.drop(droplist,axis=1,inplace=True)

assert(len(df_aedes.columns) == len(cart)) # Make sure all possible KMERS exist


,sequence,class
0,cta ta a t ta taa aa a a aa aac ac c a a...,0
1,cta ta a t ta taa aa a a aa aac ac c a a...,0
2,agt gt t g gt gtt tt t t tt ttg tg g t t...,0
3,agt gt t g gt gtt tt t t tt ttg tg g t t...,0
4,atg tg g t tg tga ga a g ga gaa aa a a a...,0
...,...,...
319,AGAAGTTTATCTGTGTGAACTTCTTGGCTTAGTATCGTTGAGAAGA...,1
320,AGAAGTTTATCTGTGTGAACTTCTTGGCTTAGTATCGTTGAGAAGA...,1
321,AGAAGTTTATCTGTGTGAACTTCTTGGCTTAGTATCGTTGAGAAGA...,1
322,AGAAGTTTATCTGTGTGAACTTCTTGGCTTAGTATCGTTGAGAAGA...,1


In [ ]:
KMERS =3

def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

def getKmersStr(sequence, size):
    kmers = getKmers(sequence, size)
    return " ".join(kmers)

# Aedes is a vector
# Each vectors dna sample has the same # of kmers??
cv = CountVectorizer()

for path in paths:
    sequences = []
    ids = []
    for index, seq_record in enumerate(SeqIO.parse(path, "fasta")):
        sequences.append(getKmersStr(str(seq_record.seq), size=KMERS))
        ids.append(seq_record.id)
    
    X = cv.fit_transform(sequences).toarray() # holds the kmers and their counts
    tokens = cv.get_feature_names()
    df = pd.DataFrame(data=X, index=ids, columns=tokens)
    print(df.loc[:, (df != 0).any(axis=0)])
    print("Finished path {}".format(path))
    
    import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from Bio import SeqIO # FASTA reader

path = "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Aedes/Dengue_virus.fasta"



# Aedes is a vector
# Each vectors dna sample has the same # of kmers??

sequences = []
ids = []
for index, seq_record in enumerate(SeqIO.parse(path, "fasta")):
    if ("aab" in str(seq_record.seq)):
        print(seq_record.id)
    sequences.append(getKmersStr(str(seq_record.seq), size=KMERS))
    ids.append(seq_record.id)

print("Finished path {}".format(path))

In [ ]:
df = df.loc[:, (df != 0).any(axis=0)]
print(df)


df = df.loc[:, df.isin([' ','NULL',0]).mean() < 1]
print (df)
# iterating the columns 
for col in df.columns: 
    print(col)

In [ ]:
def toString(List): 
    return ''.join(List) 
  
# Function to print permutations of string 
# This function takes three parameters: 
# 1. String 
# 2. Starting index of the string 
# 3. Ending index of the string. 
def permute(a, l, r): 
    if l==r: 
        print (toString(a) )
    else: 
        for i in range(l,r+1): 
            a[l], a[i] = a[i], a[l] 
            permute(a, l+1, r) 
            a[l], a[i] = a[i], a[l] # backtrack 
  
# Driver program to test the above function 
string = "acgt"
n = len(string) 
a = list(string) 
permute(a, 0, n-1) 
  

In [ ]:
cols = []
for col in df.columns: 
    print(col)
    cols.append(col)

In [ ]:
cols == cart # all possible combinations are shown which is good
# if there was a kmer which was not found it might have accidentally been dropped earlier!

In [ ]:
print (df)

In [ ]:
df.value_counts().sort_index().plot.bar()


***************

In [39]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from Bio import SeqIO # FASTA reader
import numpy as np

path = "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Aedes/Dengue_virus.fasta"



In [42]:
sequences
classes = np.full(shape=len(sequences),fill_value=0)


5446
5446


In [37]:
a = np.array([1,2,3,4,5])
b = np.array([6,7,8,9,10])
np.array(list(zip(a,b)))


array([[ 1,  6],
       [ 2,  7],
       [ 3,  8],
       [ 4,  9],
       [ 5, 10]])

In [41]:
numpy_data = np.array(list(zip(sequences,classes)))
df_aedes = pd.DataFrame(data=numpy_data)
df_aedes.columns = ["sequence","class"]
#({"columns":})
df_aedes.head()

,sequence,class
0,CTAACAGTTTTTTATTAGAGAGCAGATCTCTGATGAACAACCAACG...,0
1,CTAACAGTTTTTTATTAGAGAGCAGATCTCTGATGAACAACCAACG...,0
2,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGAATCGGAAG...,0
3,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGACTCGGAAG...,0
4,ATGAATAACCAACGAAAAAAGGCGAGAAATACGCCTTTCAATATGC...,0


In [45]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from Bio import SeqIO # FASTA reader
import numpy as np

path = "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Culex/Japanese_encephalitis_virus.fasta"

# Get Sequences
sequences = []
ids = []
for index, seq_record in enumerate(SeqIO.parse(path, "fasta")):
    sequences.append(str(seq_record.seq))
    ids.append(seq_record.id)                 

In [46]:
classes = np.full(shape=len(sequences),fill_value=1)
print(len(classes))
print(len(sequences))

324
324


In [47]:
numpy_data = np.array(list(zip(sequences,classes)))
df_culex = pd.DataFrame(data=numpy_data)
df_culex.columns = ["sequence","class"]
#({"columns":})
df_culex.head()

,sequence,class
0,AGAAGTTTATCTGTGTGGACTTCTTGGCTTAGTATCGTTGAGAAGA...,1
1,AGAAGTTTATCTGTGTGGACTTCTTGGCTTAGTATCGTTGAGAAGA...,1
2,AGAAGTTTATCTGTGTGAACTTCTTGGCTTAGTATCGTTGAGAAGA...,1
3,AGAAGTTTATCTGTGTGAACTTCTTGGCTTAGTATCGTTGAGAAGA...,1
4,AGAAGTTTATCTGTGTGAACTTCTTGGCTTAGTATCGTTGAGAAGA...,1


In [57]:
frames = [df_aedes, df_culex]

full_df = df_aedes.append(df_culex)


In [58]:
full_df.tail()
full_df.shape

(5770, 2)

In [59]:
df_aedes.tail()


,sequence,class
5441,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGAATCGGAAG...,0
5442,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGAATCGGAAG...,0
5443,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGAATCGGAAG...,0
5444,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGAATCGGAAG...,0
5445,GTGGACCGCAAAGAACAGTTTCGAATCGGAAGCTTGCTTAACGTAG...,0


In [1]:
df_aedes.shape
df_aedes.new_shape(324,2)
df_aedes.shape

NameError: name 'df_aedes' is not defined

In [60]:
df_culex.shape

(324, 2)

In [ ]:
                 

KMERS = 3

def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

def getKmersStr(sequence, size):
    kmers = getKmers(sequence, size)
    return " ".join(kmers)

# Aedes is a vector
# Each vectors dna sample has the same # of kmers??
cv = CountVectorizer()



X = cv.fit_transform(sequences).toarray() # holds the kmers and their counts
tokens = cv.get_feature_names()
df = pd.DataFrame(data=X, index=ids, columns=tokens)
df = df.loc[:, (df != 0).any(axis=0)]


print("Finished path {}".format(path))